<a href="https://colab.research.google.com/github/ayaabdo/Image-Paragraph-Captioning/blob/master/IPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Let's fight!")

Let's fight!
